### Step 1. 데이터 다운로드

Cloud Shell 통해 ~/aiffel/lyricist/data/lyrics 경로에 데이터 두었음.

### Step 2. 데이터 읽어오기

In [1]:
import glob
import os
import re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### Step 3. 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뛰기
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뛰기.(화자가 표기된 문장)

    if idx > 9: break   #문장 10개만 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자변환, 양쪽 공백 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽 공백 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 여러개->한개로 변환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 해당하지 않는 모든 문자를 공백으로 바꿈
    sentence = sentence.strip() #양쪽 공백 지움
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작과 끝에 <start>, <end> 각각 추가
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을 것
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장 건너뛰기
    if len(sentence) == 0: continue
    if len(sentence) > 15: continue #토큰 개수가 15개 이상이면 학습데이터에서 제외
    if sentence[-1] == ":": continue
    
    # 정제하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제결과 10개 확인
corpus[:10]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> and even though <end>',
 '<start> hallelujah <end>']

In [5]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer 만들기
    # 12000어에 포함되지 못한 단어는 '<unk>'로 바꿈
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이 맞춰주기
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 162   3 ...   0   0   0]
 [  2 162   3 ...   0   0   0]
 [  2  56 881 ...   0   0   0]
 ...
 [  2 216 554 ...   0   0   0]
 [  2  60  17 ...   0   0   0]
 [  2  36  21 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f307f48d940>


In [6]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만들기
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

### Step 4. 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, 
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=10)

### Step 5. 인공지능 만들기

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

# 불러온 데이터를 모델에 넣어보기
lyricist(src_sample)

<tf.Tensor: shape=(256, 12, 12001), dtype=float32, numpy=
array([[[ 1.43329544e-05, -3.20915206e-05, -2.48686527e-04, ...,
          1.12780850e-04, -2.01628762e-04, -1.31755383e-04],
        [ 1.12791684e-04, -3.89024732e-04, -4.39185882e-04, ...,
          2.69203418e-04, -3.67890607e-04, -1.59288829e-04],
        [ 1.80044619e-04, -1.00478542e-03, -5.10822516e-04, ...,
          3.15218727e-04, -4.97448200e-04,  9.40282735e-06],
        ...,
        [ 1.10952661e-03, -1.16005365e-03,  8.27888798e-05, ...,
         -2.49444973e-03, -1.75822363e-03, -1.62244588e-03],
        [ 1.22946594e-03, -1.06495630e-03,  2.59604945e-04, ...,
         -2.76751793e-03, -1.96622987e-03, -1.88925315e-03],
        [ 1.33414532e-03, -9.69625486e-04,  4.27857391e-04, ...,
         -2.99144699e-03, -2.16275291e-03, -2.14671949e-03]],

       [[ 1.43329544e-05, -3.20915206e-05, -2.48686527e-04, ...,
          1.12780850e-04, -2.01628762e-04, -1.31755383e-04],
        [ 1.56271257e-04, -8.32891237e-05, -4

In [11]:
lyricist.summary()
#우리 모델은 입력 시퀀스의 길이를 모르기 때문에 output shape를 특정할 수 없다.

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
55/55 [==============================] - 10s 152ms/step - loss: 2.8003
Epoch 2/10
55/55 [==============================] - 9s 159ms/step - loss: 1.6483
Epoch 3/10
55/55 [==============================] - 9s 154ms/step - loss: 1.4503
Epoch 4/10
55/55 [==============================] - 8s 149ms/step - loss: 1.3967
Epoch 5/10
55/55 [==============================] - 8s 145ms/step - loss: 1.3682
Epoch 6/10
55/55 [==============================] - 8s 143ms/step - loss: 1.3471
Epoch 7/10
55/55 [==============================] - 8s 142ms/step - loss: 1.3240
Epoch 8/10
55/55 [==============================] - 8s 142ms/step - loss: 1.2968
Epoch 9/10
55/55 [==============================] - 8s 144ms/step - loss: 1.2699
Epoch 10/10
55/55 [==============================] - 8s 145ms/step - loss: 1.2402


### Step 5. 인공지능 만들기

In [13]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it <end> '

간결하고 임팩트있는 한 문장이 나왔습니다.